In [1]:
from sensor.utils.main_utils import read_yaml_file,write_yaml_file
import pandas as pd
import  numpy as np
from sensor.logger import logging
import os

C:\Users\Anil Jambhulkar\AppData\Local\Temp\ipykernel_18284\3049780951.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.DataFrame([[1,2],[1,2]])

In [3]:
df

,0,1
0,1,2
1,1,2


In [4]:
pd.concat([df,df])

,0,1
0,1,2
1,1,2
0,1,2
1,1,2


In [5]:
file_path=r"C:\Users\Anil Jambhulkar\Desktop\Sensor-Fault-Detection\config\schema.yaml"

In [6]:
schema_config = read_yaml_file(filepath=file_path) 

In [7]:
len(schema_config['columns'])

164

In [8]:
len(df.columns)

2

In [9]:
len(schema_config['columns']) == len(df.columns)

False

In [10]:
from scipy.stats import ks_2samp

P-value for exact same elements 

In [11]:
d1 = np.arange(10)
d2 = np.arange(10)

In [12]:
ks_2samp(d1,d2)

KstestResult(statistic=0.0, pvalue=1.0)

Distinct dataset values

In [13]:
d1 = np.arange(100,200)
d2 = np.arange(10)

In [14]:
d1

array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
       152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
       165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
       178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
       191, 192, 193, 194, 195, 196, 197, 198, 199])

In [15]:
d2

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [16]:
ks_2samp(d1,d2)

KstestResult(statistic=1.0, pvalue=4.2646072253826637e-14)

In [17]:
test = ks_2samp(d1,d2)

In [18]:
round(test.pvalue)

0

In [19]:
training_file_path = r"C:\Users\Anil Jambhulkar\Desktop\Sensor-Fault-Detection\artifact\03_02_2024_15_46_14\data_ingestion\ingested\train.csv"

In [20]:
test_file_path = r"C:\Users\Anil Jambhulkar\Desktop\Sensor-Fault-Detection\artifact\03_02_2024_15_46_14\data_ingestion\ingested\test.csv" 

In [21]:
train_df = pd.read_csv(training_file_path)
test_df = pd.read_csv(test_file_path)

In [22]:
train_df.columns

Index(['class', 'aa_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000',
       'ag_001', 'ag_002', 'ag_003',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=164)

In [23]:
test_df.columns

Index(['class', 'aa_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000',
       'ag_001', 'ag_002', 'ag_003',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=164)

In [24]:
def get_drift_report(base_df,current_df,threshold=0.05)->bool:
    report = {}
    for column in base_df.columns:
        d1 = base_df[column]
        d2 = current_df[column]
        # Check if the column data type is numeric
        if np.issubdtype(d1.dtype, np.number) and np.issubdtype(d2.dtype, np.number):
            # Perform drift detection if both columns have numeric data types
            is_same_dist = ks_2samp(d1, d2)
            if threshold <= is_same_dist.pvalue:
                is_found = False
            else:
                is_found = True
                status = False
            report.update({
                column: {
                    "p_value": float(is_same_dist.pvalue),
                    "drift_status": is_found
                }
            })
        else:
            # Skip drift detection if either column has a non-numeric data type
            logging.warning(f"Skipping drift detection for non-numeric column '{column}'")       
    return report

In [25]:
drift_report = get_drift_report(train_df,test_df)

In [26]:
drift_report

{'aa_000': {'p_value': 0.9089534436058394, 'drift_status': False},
 'ac_000': {'p_value': 0.13205762704470103, 'drift_status': False},
 'ad_000': {'p_value': 0.7450166398394409, 'drift_status': False},
 'ae_000': {'p_value': 0.9892162247324442, 'drift_status': False},
 'af_000': {'p_value': 0.9899427818708384, 'drift_status': False},
 'ag_000': {'p_value': 1.0, 'drift_status': False},
 'ag_001': {'p_value': 1.0, 'drift_status': False},
 'ag_002': {'p_value': 0.9977825627750571, 'drift_status': False},
 'ag_003': {'p_value': 0.9941111269293684, 'drift_status': False},
 'ag_004': {'p_value': 0.9132444186708828, 'drift_status': False},
 'ag_005': {'p_value': 0.6637191654102459, 'drift_status': False},
 'ag_006': {'p_value': 0.7149969934437623, 'drift_status': False},
 'ag_007': {'p_value': 0.29914929584056804, 'drift_status': False},
 'ag_008': {'p_value': 0.6660723934855366, 'drift_status': False},
 'ag_009': {'p_value': 0.9190646828820624, 'drift_status': False},
 'ah_000': {'p_value': 

In [27]:
len(drift_report)

163

In [28]:
write_yaml_file(file_path=os.path.join(os.getcwd(),"report.yaml"),content=drift_report,replace=True)

In [29]:
# def get_drift_report(base_df,current_df,threshold=0.05)->bool:
#     report = {}
#     for column in base_df.columns:
#         d1 = base_df[column]
#         d2 = current_df[column]
#         # Check if the column data type is numeric
#         if np.issubdtype(d1.dtype, np.number) and np.issubdtype(d2.dtype, np.number):
#             # Perform drift detection if both columns have numeric data types
#             is_same_dist = ks_2samp(d1, d2)
#             if threshold <= is_same_dist.pvalue:
#                 is_found = False
#             else:
#                 is_found = True
#                 status = False
#             report.update({
#                 column: {
#                     "p_value": float(is_same_dist.pvalue),
#                     "drift_status": is_found
#                 }
#             })
#         else:
#             # Skip drift detection if either column has a non-numeric data type
#             logging.warning(f"Skipping drift detection for non-numeric column '{column}'")       
#     return report

In [30]:
from sensor.ml.model.estimator import TargetValueMapping

In [31]:
TargetValueMapping().to_dict()

{'neg': 0, 'pos': 1}

In [32]:
TargetValueMapping().reverse_mapping()

{0: 'neg', 1: 'pos'}